In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
directory = os.getcwd()
data = '/data/vectors/data0625/'
path = directory + data

In [3]:
fft = 2048

In [4]:
data = np.empty((0,fft+1))
for fname in ['wp.txt','nowp.txt','silence.txt']:
    print(fname)
    with open(path+fname) as f:
        dat = np.genfromtxt(f, delimiter=',', dtype='int', 
                            invalid_raise=False)
        if fname.startswith('w'):
            dat[:,-1] = 2
        elif fname.startswith('no'):
            dat[:,-1] = 1
        else:
            dat[:,-1] = 0 ## silence
    data = np.concatenate([data,dat],axis=0)     

wp.txt
nowp.txt
silence.txt


In [5]:
data.shape

(1308, 2049)

In [6]:
data_set = data.copy()

## Split train/validation/test

In [7]:
train_len = int(.6*(data_set.shape[0]))
val_len = int(.2*(data_set.shape[0]))
np.random.seed(21)
np.random.shuffle(data_set)
trainNN, valNN, testNN = np.split(data_set, [train_len,train_len+val_len])

X_trainNN = trainNN[:,:-1]
y_trainNN = trainNN[:,-1]
X_valNN = valNN[:,:-1]
y_valNN = valNN[:,-1]
X_testNN = testNN[:,:-1]
y_testNN = testNN[:,-1]

print('number of sample: '+ str(len(X_trainNN)))
print('number of sample: '+ str(len(y_valNN)))
print('number of sample: '+ str(len(y_testNN)))

number of sample: 784
number of sample: 261
number of sample: 263


## modeling

In [8]:
tf.keras.backend.clear_session()

In [9]:
model = keras.Sequential()
model.add(keras.Input(shape=(fft,)))
model.add(keras.layers.Dropout(.1))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dropout(.1))
model.add(keras.layers.Dense(12, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [10]:
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['acc'])

In [11]:
best_model_path = './models/TFLite/0625/threeClass/NoQAW.h5'

es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                   mode='min', 
                                   verbose=0, 
                                   patience=10)
mc = keras.callbacks.ModelCheckpoint(filepath=best_model_path, 
                                     monitor='val_acc', 
                                     mode='max', 
                                     verbose=0, 
                                     save_best_only=True)


In [12]:
history = model.fit(X_trainNN,
                    y_trainNN, 
                    epochs=500,
                    batch_size=40,
                    validation_data=(X_valNN, y_valNN),
                    callbacks=[mc,es])

Epoch 1/500
20/20 [==============================] - 0s 11ms/step - loss: 1.6051 - acc: 0.4758 - val_loss: 0.5253 - val_acc: 0.7318
Epoch 2/500
20/20 [==============================] - 0s 2ms/step - loss: 0.5732 - acc: 0.7079 - val_loss: 0.4944 - val_acc: 0.6437
Epoch 3/500
20/20 [==============================] - 0s 2ms/step - loss: 0.4441 - acc: 0.7462 - val_loss: 0.4341 - val_acc: 0.6897
Epoch 4/500
20/20 [==============================] - 0s 2ms/step - loss: 0.4359 - acc: 0.7640 - val_loss: 0.4172 - val_acc: 0.7165
Epoch 5/500
20/20 [==============================] - 0s 3ms/step - loss: 0.4224 - acc: 0.7755 - val_loss: 0.3941 - val_acc: 0.7893
Epoch 6/500
20/20 [==============================] - 0s 3ms/step - loss: 0.3585 - acc: 0.8253 - val_loss: 0.3519 - val_acc: 0.8506
Epoch 7/500
20/20 [==============================] - 0s 3ms/step - loss: 0.3681 - acc: 0.8380 - val_loss: 0.3132 - val_acc: 0.8774
Epoch 8/500
20/20 [==============================] - 0s 3ms/step - loss: 0.3103 - 

In [13]:
test_loss = model.evaluate(X_testNN, y_testNN)

9/9 [==============================] - 0s 666us/step - loss: 0.0771 - acc: 0.9620


In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_testNN, np.argmax(model.predict(X_testNN), axis=-1))

array([[97,  0,  0],
       [ 0, 79,  2],
       [ 0,  8, 77]])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                65568     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 12)                396       
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 39        
Total params: 66,003
Trainable params: 66,003
Non-trainable params: 0
_________________________________________________________________


## Full Integer Post-Model Quantization

In [16]:
def representative_data_gen():
    for dat in trainNN: 
        dat = dat.astype('float32')
        yield [dat[:fft]]

In [17]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.int8]

# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.int8
#converter.inference_output_type = tf.uint8

# And this sets the representative dataset so we can quantize the activations
converter.representative_dataset = representative_data_gen
quantized_tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /var/folders/7m/7gjw27ys7vs7gkgvbdgx5rjc0000gn/T/tmpwcqs9p7v/assets


In [18]:
filename = './models/TFLite/0625/threeClass/NoQAW_Quantized.tflite'
with open(filename, 'wb') as f:
    f.write(quantized_tflite_model)

In [19]:
model_size = os.path.getsize(best_model_path)
print("Original model is %d KB" % int(model_size/1000))

quantized_model_size = os.path.getsize(filename)
print("Quantized model is %d KB" % int(quantized_model_size/1000))

Original model is 824 KB
Quantized model is 68 KB


## Validate the Converted Model

In [20]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=filename) 
interpreter.get_tensor_details()

[{'name': 'input_1',
  'index': 0,
  'shape': array([   1, 2048], dtype=int32),
  'shape_signature': array([  -1, 2048], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.45490196347236633, -128),
  'quantization_parameters': {'scales': array([0.45490196], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential/dense/BiasAdd/ReadVariableOp/resource',
  'index': 1,
  'shape': array([32], dtype=int32),
  'shape_signature': array([32], dtype=int32),
  'dtype': numpy.int32,
  'quantization': (0.0005456146318465471, 0),
  'quantization_parameters': {'scales': array([0.00054561], dtype=float32),
   'zero_points': array([0], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'sequential/dense_1/BiasAdd/ReadVariableOp/resource',
  'index': 2,
  'shape': array([12], dtype=int32),
  'shape_signature': array([12], dtype=int32),
  'dtype': numpy.int32,
  'quantization': 

## Inference using Full-Quantized TFLite Model

In [21]:
interpreter = tf.lite.Interpreter(model_content=quantized_tflite_model)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
scale, zero_points = input_details['quantization']

tflite_predict = []
tflite_predict_class = []
for vector in X_testNN:
    input_data = np.array(vector.reshape(input_details['shape'])/scale + zero_points, dtype=np.int8)
    interpreter.set_tensor(input_details['index'], input_data)
    
    interpreter.invoke()    
    prediction = interpreter.get_tensor(output_details['index'])
    tflite_predict.append(prediction)
    tflite_predict_class.append(np.argmax(prediction))

### Performance of both the model and its optimized version are similar

In [22]:
confusion_matrix(y_testNN, tflite_predict_class)

array([[97,  0,  0],
       [ 0, 79,  2],
       [ 0,  8, 77]])